In [69]:
# Setup
from huggingface_hub import hf_hub_download
from datasets import load_dataset
import os 
os.chdir("C:\\Users\\WulfN\\.cache\\huggingface\\hub\\")

%config InteractiveShell.ast_node_interactivity = 'all'
# download model
#hf_hub_download(repo_id = 'prithivMLmods/Deep-Fake-Detector-Model', filename = 'config.json') # presumably the config file is needed first
# model.safetensors, # pytorch_model.bin

# Datasets can be found here: https://huggingface.co/datasets?modality=modality:image&sort=trending&search=deepfake



In [2]:
### Load Data
dataset = load_dataset("Whab/deepfake")

# Preview dataset images

In [3]:
#import ipywidgets as widgets
from IPython.display import display
import ipyplot
#from PIL import Image
#import matplotlib.pyplot as plt

preview_num = 5

sample_images = dataset['train'][1:preview_num + 1]

# display_images = [
#     widgets.Image(value=sample_images['image'][i].tobytes(), format='png', width=256, height=256) 
#     for i in range(preview_num)
# ]

# display_images
# display(widgets.HBox(display_images))

## Display images vertically
# for i in range(preview_num):
#     display(sample_images['image'][i])

# display(display_images) 
# #widgets.HBox(display_images)
# # hbox = widgets.HBox(display_images)
# # display(hbox)
# #display_images

In [26]:
### Pipeline approach
from transformers import pipeline

pipe = pipeline("image-classification", model="prithivMLmods/Deep-Fake-Detector-Model", device=-1);  

### Load model directly:
# from transformers import AutoImageProcessor, AutoModelForImageClassification

# processor = AutoImageProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")
# model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")

Function to reate batches of image data

In [ ]:
import pandas as pd
# import modin as pd
# add multiprocessing: https://superfastpython.com/multiprocessing-for-loop/

def create_image_batches(data, batch_size):
    
    num_images = data['train'].num_rows
    batch_size = batch_size
    # index value of first image in batch i
    batch_key = list(range(0, num_images, batch_size))
    
    result = pd.DataFrame({'index': [],
                           'image_batch':[]})
    
    for i in range(len(batch_key)):

        # index of all images in batch
        batch_indices = list(range(batch_key[i], batch_key[i] + batch_size))

        # appending images from batch into list
        batch_i = []
        for j in batch_indices:
            add_image = data['train'][j]['image']
            batch_i.append(add_image) 

        batch = pd.DataFrame({'index': [i],
                              'image_batch': [batch_i]})
        
        # Adding batch to dataframe
        result = pd.concat([result, batch])


In [ ]:
batches = create_image_batches(dataset, batch_size = 3)

# time with pandas

In [ ]:
print('Number of batches: 'len(batches))
batches.head(20)

In [ ]:
import pandas as pd
#import psutil
# consider doing different values of i 

df_length = 100

# Rather than pipe(image i), pipe(batch of images i), try batch of 3 images then 2
predictions = pd.DataFrame()
for i in range(df_length):
    image_score = pd.DataFrame(pipe(dataset['train'][:df_length+1]['image'][i])) #dataset['train'][:df_length+1]['image'][i]
    predictions = pd.concat([predictions, pd.concat({i: image_score}, names=['image_num'])])

predictions
# AUC score

label     score
image_num                  
0         0  Fake  0.976109
          1  Real  0.020260
1         0  Fake  0.978068
          1  Real  0.018317
2         0  Fake  0.977322
...           ...       ...
997       1  Real  0.020522
998       0  Fake  0.973141
          1  Real  0.022830
999       0  Fake  0.978275
          1  Real  0.017491

[2000 rows x 2 columns]

In [ ]:
# Save table in repo
import pickle
predictions.to_pickle('huggingface_predictions')

dist'n of fake and real scores

In [ ]:
# dist'n of fake and real scores
import seaborn as sb
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Fake'], bins=100)
sb.displot(kind='hist', data=predictions.loc[lambda x: x.label == 'Real'], bins=100)

In [ ]:
# AUC score